**INTRO**

In [1]:
!mkdir -p scripts logs data/raw data/processed db config


In [2]:
%%writefile scripts/__init__.py
# makes 'scripts' a package so imports resolve properly


Writing scripts/__init__.py


**PHASE 2 : EXTRACT**

create scripts/extract.py

In [3]:
%%writefile scripts/extract.py
import pandas as pd
import logging
from datetime import datetime

# configure logging -> runs when the script is imported
logging.basicConfig(
    filename="logs/pipeline.log",
    level=logging.INFO,
    format="%(asctime)s - [%(levelname)s] - %(message)s"
)


def extract_data(url: str, sample_rows: int = None) -> pd.DataFrame:
    logging.info("Starting data extraction...")
    try:
        df = pd.read_csv(url)

        if sample_rows:
            df = df.head(sample_rows)
            logging.info(f"Dataset sampled to {sample_rows} rows.")

        logging.info(f"Extraction successful. Loaded {len(df)} rows and {len(df.columns)} columns.")
        return df

    except Exception as e:
        logging.error(f"Extraction failed: {e}")
        raise

Writing scripts/extract.py


load dataset

In [4]:
from scripts.extract import extract_data

url = url = "https://raw.githubusercontent.com/GurpartapG/etl-dataflow-demo/main/data/raw/Airline_Delay_Cause.csv"
df = extract_data(url)

**PHASE 3 : TRANSFORM**




check the dataset

In [5]:
# load sample data
df.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2025,7,YV,Mesa Airlines Inc.,BWI,"Baltimore, MD: Baltimore/Washington Internatio...",18.0,2.0,1.43,0.00,...,0.0,0.00,0.0,0.0,44.0,32.0,0.0,12.0,0.0,0.0
1,2025,7,YV,Mesa Airlines Inc.,CHS,"Charleston, SC: Charleston AFB/International",48.0,16.0,4.06,3.15,...,0.0,3.30,2.0,0.0,1282.0,331.0,438.0,259.0,0.0,254.0
2,2025,7,YV,Mesa Airlines Inc.,CLE,"Cleveland, OH: Cleveland-Hopkins International",65.0,10.0,3.53,1.00,...,0.0,2.43,2.0,0.0,735.0,151.0,172.0,139.0,0.0,273.0
3,2025,7,YV,Mesa Airlines Inc.,CLT,"Charlotte, NC: Charlotte Douglas International",134.0,31.0,13.19,2.43,...,0.0,4.74,8.0,0.0,2264.0,798.0,303.0,702.0,0.0,461.0
4,2025,7,YV,Mesa Airlines Inc.,CMH,"Columbus, OH: John Glenn Columbus International",61.0,12.0,6.44,0.28,...,0.0,1.69,0.0,0.0,630.0,211.0,17.0,123.0,0.0,279.0


In [6]:
# show cols
df.columns.tolist()

['year',
 'month',
 'carrier',
 'carrier_name',
 'airport',
 'airport_name',
 'arr_flights',
 'arr_del15',
 'carrier_ct',
 'weather_ct',
 'nas_ct',
 'security_ct',
 'late_aircraft_ct',
 'arr_cancelled',
 'arr_diverted',
 'arr_delay',
 'carrier_delay',
 'weather_delay',
 'nas_delay',
 'security_delay',
 'late_aircraft_delay']

In [7]:
# view datatypes and non null info for each col
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13295 entries, 0 to 13294
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   year                 13295 non-null  int64  
 1   month                13295 non-null  int64  
 2   carrier              13295 non-null  object 
 3   carrier_name         13295 non-null  object 
 4   airport              13295 non-null  object 
 5   airport_name         13295 non-null  object 
 6   arr_flights          13292 non-null  float64
 7   arr_del15            13289 non-null  float64
 8   carrier_ct           13292 non-null  float64
 9   weather_ct           13292 non-null  float64
 10  nas_ct               13292 non-null  float64
 11  security_ct          13292 non-null  float64
 12  late_aircraft_ct     13292 non-null  float64
 13  arr_cancelled        13292 non-null  float64
 14  arr_diverted         13292 non-null  float64
 15  arr_delay            13292 non-null 

In [8]:
# check for missing valuees
df.isna().sum()

,0
year,0
month,0
carrier,0
carrier_name,0
airport,0
airport_name,0
arr_flights,3
arr_del15,6
carrier_ct,3
weather_ct,3


In [9]:
df[df.isna().any(axis=1)][[
    "year", "month", "carrier", "airport",
    "arr_flights", "arr_del15", "arr_delay"
]]

,year,month,carrier,airport,arr_flights,arr_del15,arr_delay
301,2025,7,MQ,MKE,1.0,NaN,0.0
2811,2025,6,MQ,DAY,NaN,NaN,NaN
3785,2025,6,YX,ELP,NaN,NaN,NaN
4250,2025,5,C5,OKC,1.0,NaN,0.0
6122,2025,4,OO,CAE,NaN,NaN,NaN
10256,2025,2,G7,DAY,1.0,NaN,0.0


- if arr_flights = NaN, no flight was recorded
- if arr_flights > 0, that means the flight existed but not delayed

create scripts/transform.py

In [10]:
%%writefile scripts/transform.py
import pandas as pd
import logging

# configure logging for this module
logging.basicConfig(
    filename="logs/pipeline.log",
    level=logging.INFO,
    format="%(asctime)s - [%(levelname)s] - %(message)s"
)

def transform_data(df: pd.DataFrame) -> pd.DataFrame:

    logging.info("Starting data transformation...")
    try:

      # ensure all numeric count cols are integers since these represent count of flights
      count_cols = [
          "arr_flights", "arr_del15", "arr_cancelled", "arr_diverted"
      ]

      # convert type
      for col in count_cols:
        # use Int64 in case there are any missing values
        df[col] = df[col].fillna(0).astype("Int64")


      # avoid division by zero for rate calculation
      flights_nonzero = df["arr_flights"].replace(0, pd.NA)

      # feature engineer the rates
      df["delay_rate"] = df["arr_del15"] / flights_nonzero
      df["cancel_rate"] = df["arr_cancelled"] / flights_nonzero
      df["divert_rate"] = df["arr_diverted"] / flights_nonzero

      # feature engineer new date col
      df["date"] = pd.to_datetime(df[["year", "month"]].assign(day=1))

      # fill missing rate with 0
      for col in ["delay_rate", "cancel_rate", "divert_rate"]:
        df[col] = df[col].fillna(0)

      # delay category based on delay rates
      # 0-5% -> good
      # 5-10% -> moderate
      # 10%+ -> excellent
      df["delay_category"] = pd.cut(
          df["delay_rate"],
          bins = [-0.01, 0.05, 0.10, 1.0],
          labels = ["good", "moderate", "poor"]
      )


      # list of all delay/count/duration columns
      delay_cols = [
      "arr_del15", "arr_cancelled", "arr_diverted",
      "arr_delay", "carrier_ct", "weather_ct", "nas_ct",
      "security_ct", "late_aircraft_ct",
      "carrier_delay", "weather_delay", "nas_delay",
      "security_delay", "late_aircraft_delay"
      ]

      # case a: flights == 0 → set everything to 0
      df.loc[df["arr_flights"] == 0, delay_cols] = 0
      # case B: flights > 0 but delay values are missing → treat as 0
      df[delay_cols] = df[delay_cols].fillna(0)


      logging.info("Transformation completed successfully.")
      return df

    except Exception as e:
      logging.error(f"Transformation failed: {e}")
      raise

Writing scripts/transform.py


use transform function

In [11]:
from scripts.transform import transform_data

df_transformed = transform_data(df.copy())

sanity checks

In [12]:
# check new dataset
df_transformed.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,delay_rate,cancel_rate,divert_rate,date,delay_category
0,2025,7,YV,Mesa Airlines Inc.,BWI,"Baltimore, MD: Baltimore/Washington Internatio...",18,2,1.43,0.00,...,32.0,0.0,12.0,0.0,0.0,0.111111,0.0,0.0,2025-07-01,poor
1,2025,7,YV,Mesa Airlines Inc.,CHS,"Charleston, SC: Charleston AFB/International",48,16,4.06,3.15,...,331.0,438.0,259.0,0.0,254.0,0.333333,0.041667,0.0,2025-07-01,poor
2,2025,7,YV,Mesa Airlines Inc.,CLE,"Cleveland, OH: Cleveland-Hopkins International",65,10,3.53,1.00,...,151.0,172.0,139.0,0.0,273.0,0.153846,0.030769,0.0,2025-07-01,poor
3,2025,7,YV,Mesa Airlines Inc.,CLT,"Charlotte, NC: Charlotte Douglas International",134,31,13.19,2.43,...,798.0,303.0,702.0,0.0,461.0,0.231343,0.059701,0.0,2025-07-01,poor
4,2025,7,YV,Mesa Airlines Inc.,CMH,"Columbus, OH: John Glenn Columbus International",61,12,6.44,0.28,...,211.0,17.0,123.0,0.0,279.0,0.196721,0.0,0.0,2025-07-01,poor


In [13]:
# check col info
df_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13295 entries, 0 to 13294
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   year                 13295 non-null  int64         
 1   month                13295 non-null  int64         
 2   carrier              13295 non-null  object        
 3   carrier_name         13295 non-null  object        
 4   airport              13295 non-null  object        
 5   airport_name         13295 non-null  object        
 6   arr_flights          13295 non-null  Int64         
 7   arr_del15            13295 non-null  Int64         
 8   carrier_ct           13295 non-null  float64       
 9   weather_ct           13295 non-null  float64       
 10  nas_ct               13295 non-null  float64       
 11  security_ct          13295 non-null  float64       
 12  late_aircraft_ct     13295 non-null  float64       
 13  arr_cancelled        13295 non-

In [14]:
df_transformed.isna().sum()

,0
year,0
month,0
carrier,0
carrier_name,0
airport,0
airport_name,0
arr_flights,0
arr_del15,0
carrier_ct,0
weather_ct,0


**PHASE 4 : LOAD**

Schema Definition:

CREATE TABLE IF NOT EXISTS flight_delays (
    
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    year INTEGER,
    month INTEGER,
    carrier TEXT,
    carrier_name TEXT,
    airport TEXT,
    airport_name TEXT,
    arr_flights INTEGER,
    arr_del15 INTEGER,
    arr_cancelled INTEGER,
    arr_diverted INTEGER,
    arr_delay REAL,
    carrier_ct REAL,
    weather_ct REAL,
    nas_ct REAL,
    security_ct REAL,
    late_aircraft_ct REAL,
    carrier_delay REAL,
    weather_delay REAL,
    nas_delay REAL,
    security_delay REAL,
    late_aircraft_delay REAL,
    delay_rate REAL,
    cancel_rate REAL,
    divert_rate REAL,
    delay_category TEXT,
    date TEXT
);

Indexes:

CREATE INDEX idx_carrier_month ON flight_delays (carrier, year, month);

CREATE INDEX idx_airport_date ON flight_delays
(airport, date);


---



create load.py

In [15]:
%%writefile scripts/load.py
import sqlite3
import pandas as pd
import logging

# configure logging for this module
logging.basicConfig(
    filename="logs/pipeline.log",
    level=logging.INFO,
    format="%(asctime)s - [%(levelname)s] - %(message)s"
)

def load_to_sqlite(df: pd.DataFrame, db_path: str = "db/flights.db", table_name: str = "flight_delays"):
  logging.info("Starting data loading...")
  try:
    # connect to db
    conn = sqlite3.connect(db_path)

    # wipe and recreate using pandas
    df.to_sql(table_name, conn, if_exists="replace", index=False)

    # reapple schema
    with open("config/schema.sql", "r") as f:
      conn.executescript(f.read())

    logging.info("Load completed successfully")
    conn.close()

  except Exception as e:
    logging.error(f"Load failed: {e}")
    raise


Writing scripts/load.py


create schema

In [16]:
%%writefile config/schema.sql

CREATE TABLE IF NOT EXISTS flight_delays (

    id INTEGER PRIMARY KEY AUTOINCREMENT,
    year INTEGER,
    month INTEGER,
    carrier TEXT,
    carrier_name TEXT,
    airport TEXT,
    airport_name TEXT,
    arr_flights INTEGER,
    arr_del15 INTEGER,
    arr_cancelled INTEGER,
    arr_diverted INTEGER,
    arr_delay REAL,
    carrier_ct REAL,
    weather_ct REAL,
    nas_ct REAL,
    security_ct REAL,
    late_aircraft_ct REAL,
    carrier_delay REAL,
    weather_delay REAL,
    nas_delay REAL,
    security_delay REAL,
    late_aircraft_delay REAL,
    delay_rate REAL,
    cancel_rate REAL,
    divert_rate REAL,
    delay_category TEXT,
    date TEXT
);

CREATE INDEX IF NOT EXISTS idx_carrier_month ON flight_delays(carrier, year, month);
CREATE INDEX IF NOT EXISTS idx_airport_date ON flight_delays(airport, date);

Writing config/schema.sql


sanity checks

In [17]:
from scripts.load import load_to_sqlite
load_to_sqlite(df_transformed)

In [18]:
import sqlite3
import pandas as pd
conn = sqlite3.connect("db/flights.db")
pd.read_sql_query("SELECT * FROM flight_delays LIMIT 5", conn)

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,delay_rate,cancel_rate,divert_rate,date,delay_category
0,2025,7,YV,Mesa Airlines Inc.,BWI,"Baltimore, MD: Baltimore/Washington Internatio...",18,2,1.43,0.00,...,32.0,0.0,12.0,0.0,0.0,0.111111,0.000000,0.0,2025-07-01 00:00:00,poor
1,2025,7,YV,Mesa Airlines Inc.,CHS,"Charleston, SC: Charleston AFB/International",48,16,4.06,3.15,...,331.0,438.0,259.0,0.0,254.0,0.333333,0.041667,0.0,2025-07-01 00:00:00,poor
2,2025,7,YV,Mesa Airlines Inc.,CLE,"Cleveland, OH: Cleveland-Hopkins International",65,10,3.53,1.00,...,151.0,172.0,139.0,0.0,273.0,0.153846,0.030769,0.0,2025-07-01 00:00:00,poor
3,2025,7,YV,Mesa Airlines Inc.,CLT,"Charlotte, NC: Charlotte Douglas International",134,31,13.19,2.43,...,798.0,303.0,702.0,0.0,461.0,0.231343,0.059701,0.0,2025-07-01 00:00:00,poor
4,2025,7,YV,Mesa Airlines Inc.,CMH,"Columbus, OH: John Glenn Columbus International",61,12,6.44,0.28,...,211.0,17.0,123.0,0.0,279.0,0.196721,0.000000,0.0,2025-07-01 00:00:00,poor


In [19]:
pd.read_sql("SELECT COUNT(*) FROM flight_delays;", conn)

,COUNT(*)
0,13295


In [20]:
pd.read_sql("PRAGMA index_list(flight_delays);", conn)

,seq,name,unique,origin,partial
0,0,idx_airport_date,0,c,0
1,1,idx_carrier_month,0,c,0


**PHASE 5: ORCHESTRATION**

In [21]:
%%writefile scripts/run_pipeline.py

from scripts.extract import extract_data
from scripts.transform import transform_data
from scripts.load import load_to_sqlite
import logging
from datetime import datetime

logging.basicConfig(
    filename="logs/pipeline.log",
    level=logging.INFO,
    format="%(asctime)s - [%(levelname)s] - %(message)s"
)

def run_pipeline():
  logging.info("====== STARTING PIPELINE RUN ======")

  try:

    #extract
    FILE_URL = "https://raw.githubusercontent.com/GurpartapG/etl-dataflow-demo/main/data/raw/Airline_Delay_Cause.csv"
    df_raw = extract_data(FILE_URL)
    logging.info(f"Extract step completed. Rows: {len(df_raw)}")

    # transform
    df_transformed = transform_data(df_raw)
    logging.info(f"Transform step completed. Rows: {len(df_transformed)}")

    # load
    load_to_sqlite(df_transformed)
    logging.info(f"Load step completed")

    logging.info("====== PIPELINE COMPLETED SUCCESSFULLY ======")

  except Exception as e:
    logging.error(f"Pipeline run failed: {e}")
    raise

if __name__ == "__main__":
  run_pipeline()


Writing scripts/run_pipeline.py


In [22]:
!python -m scripts.run_pipeline

In [23]:
!cat logs/pipeline.log

2025-11-19 07:45:13,933 - [INFO] - ====== STARTING PIPELINE RUN ======
2025-11-19 07:45:13,934 - [INFO] - Starting data extraction...
2025-11-19 07:45:14,181 - [INFO] - Extraction successful. Loaded 13295 rows and 21 columns.
2025-11-19 07:45:14,181 - [INFO] - Extract step completed. Rows: 13295
2025-11-19 07:45:14,181 - [INFO] - Starting data transformation...
2025-11-19 07:45:14,302 - [INFO] - Transformation completed successfully.
2025-11-19 07:45:14,302 - [INFO] - Transform step completed. Rows: 13295
2025-11-19 07:45:14,302 - [INFO] - Starting data loading...
2025-11-19 07:45:14,920 - [INFO] - Load completed successfully
2025-11-19 07:45:14,921 - [INFO] - Load step completed
2025-11-19 07:45:14,921 - [INFO] - ====== PIPELINE COMPLETED SUCCESSFULLY ======


**PHASE 6: SQL ANALYSIS QUERIES**

In [24]:
import sqlite3, pandas as pd
conn = sqlite3.connect("db/flights.db")

In [25]:
# top 10 airlines by avg delay minutes
pd.read_sql("SELECT carrier_name, ROUND(AVG(arr_delay), 2) AS avg_delay_minutes,ROUND(AVG(delay_rate)*100, 2) AS avg_delay_rate_pct FROM flight_delays GROUP BY carrier_name HAVING COUNT(*) > 50 ORDER BY avg_delay_minutes DESC LIMIT 10;", conn)

,carrier_name,avg_delay_minutes,avg_delay_rate_pct
0,American Airlines Network,14539.39,29.64
1,Southwest Airlines,14187.06,21.99
2,Delta Air Lines Network,9316.43,22.82
3,United Air Lines Network,8935.37,24.73
4,JetBlue Airways,6098.75,26.14
5,PSA Airlines Inc.,5871.67,29.16
6,SkyWest Airlines Inc.,5090.33,22.42
7,Republic Airline,5035.42,20.76
8,Frontier Airlines,4797.06,26.59
9,Spirit Airlines,4477.64,18.50


In [26]:
# worst airports by highest delay %
pd.read_sql("SELECT airport_name, ROUND(AVG(delay_rate)*100,2) AS avg_delay_pct, COUNT(*) AS total_records FROM flight_delays GROUP BY airport_name HAVING COUNT(*) > 50 ORDER BY avg_delay_pct DESC LIMIT 10;", conn)

,airport_name,avg_delay_pct,total_records
0,"Newark, NJ: Newark Liberty International",30.83,85
1,"Denver, CO: Denver International",30.50,71
2,"Little Rock, AR: Bill and Hillary Clinton Nat ...",29.12,89
3,"Roanoke, VA: Roanoke Blacksburg Regional",28.98,57
4,"Chattanooga, TN: Lovell Field",28.37,61
5,"Huntsville, AL: Huntsville International-Carl ...",28.36,60
6,"Washington, DC: Ronald Reagan Washington National",28.34,92
7,"Columbia, SC: Columbia Metropolitan",27.91,68
8,"Greer, SC: Greenville-Spartanburg International",27.57,97
9,"Harrisburg, PA: Harrisburg International",27.54,87


In [27]:
# best airports by lowest delay %
pd.read_sql("SELECT airport_name, ROUND(AVG(delay_rate)*100,2) AS avg_delay_pct, COUNT(*) AS total_records FROM flight_delays GROUP BY airport_name HAVING COUNT(*) > 50 ORDER BY avg_delay_pct ASC LIMIT 10;", conn)

,airport_name,avg_delay_pct,total_records
0,"Santa Ana, CA: John Wayne Airport-Orange County",18.32,70
1,"Palm Springs, CA: Palm Springs International",18.58,65
2,"Salt Lake City, UT: Salt Lake City International",18.67,88
3,"San Jose, CA: Norman Y. Mineta San Jose Intern...",18.69,73
4,"Portland, OR: Portland International",18.85,83
5,"Sacramento, CA: Sacramento International",18.92,79
6,"Los Angeles, CA: Los Angeles International",19.12,90
7,"Key West, FL: Key West International",20.32,52
8,"Albuquerque, NM: Albuquerque International Sun...",20.43,78
9,"Oklahoma City, OK: Will Rogers World",20.46,56


In [28]:
# top 10 most reliable airlines with lowest delay %
pd.read_sql("SELECT carrier_name, ROUND(AVG(delay_rate)*100, 2) AS avg_delay_pct FROM flight_delays GROUP BY carrier_name HAVING COUNT(*) > 50 ORDER BY avg_delay_pct ASC LIMIT 10", conn)

,carrier_name,avg_delay_pct
0,Horizon Air,16.27
1,Spirit Airlines,18.50
2,Mesa Airlines Inc.,20.41
3,Republic Airline,20.76
4,Envoy Air,20.77
5,Piedmont Airlines,21.85
6,Southwest Airlines,21.99
7,Allegiant Air,22.04
8,SkyWest Airlines Inc.,22.42
9,Hawaiian Airlines Network,22.74


In [29]:
# delay trend by month for all airlines
pd.read_sql("SELECT year, month, ROUND(AVG(delay_rate)*100,2) AS avg_delay_pct FROM flight_delays GROUP BY year, month ORDER BY year, month;", conn)

,year,month,avg_delay_pct
0,2025,1,20.35
1,2025,2,21.96
2,2025,3,20.56
3,2025,4,20.53
4,2025,5,23.48
5,2025,6,28.64
6,2025,7,28.19


In [30]:
# delay breakdown by cause
pd.read_sql("SELECT ROUND(SUM(carrier_delay) / SUM(arr_delay) * 100, 2) AS carrier_delay_pct, ROUND(SUM(weather_delay) / SUM(arr_delay) * 100, 2) AS weather_delay_pct, ROUND(SUM(nas_delay) / SUM(arr_delay) * 100, 2) AS nas_delay_pct, ROUND(SUM(late_aircraft_delay) / SUM(arr_delay) * 100, 2) AS late_aircraft_delay_pct FROM flight_delays;", conn)

,carrier_delay_pct,weather_delay_pct,nas_delay_pct,late_aircraft_delay_pct
0,31.5,7.08,21.63,39.68


In [31]:
# where was the worst experience (airline and airport combo)
pd.read_sql("SELECT carrier_name, airport_name, ROUND(AVG(arr_delay), 2) AS avg_delay_pct, COUNT(*) AS flights FROM flight_delays GROUP BY carrier_name, airport_name  ORDER BY avg_delay_pct DESC LIMIT 10;", conn)

,carrier_name,airport_name,avg_delay_pct,flights
0,American Airlines Network,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",340088.00,7
1,Delta Air Lines Network,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",298841.00,7
2,American Airlines Network,"Charlotte, NC: Charlotte Douglas International",177201.71,7
3,SkyWest Airlines Inc.,"Denver, CO: Denver International",150030.86,7
4,United Air Lines Network,"Denver, CO: Denver International",148792.86,7
5,United Air Lines Network,"Newark, NJ: Newark Liberty International",145774.86,7
6,SkyWest Airlines Inc.,"Chicago, IL: Chicago O'Hare International",138919.14,7
7,Southwest Airlines,"Denver, CO: Denver International",125566.57,7
8,United Air Lines Network,"Chicago, IL: Chicago O'Hare International",123636.00,7
9,PSA Airlines Inc.,"Washington, DC: Ronald Reagan Washington National",118307.43,7


create queries.sql

In [32]:
%%writefile queries.sql

-- top 10 airlines by avg delay minutes
SELECT carrier_name, ROUND(AVG(arr_delay), 2) AS avg_delay_minutes,ROUND(AVG(delay_rate)*100, 2) AS avg_delay_rate_pct
FROM flight_delays
GROUP BY carrier_name
HAVING COUNT(*) > 50
ORDER BY avg_delay_minutes DESC
LIMIT 10;

-- worst airports by highest delay %
SELECT airport_name, ROUND(AVG(delay_rate)*100,2) AS avg_delay_pct, COUNT(*) AS total_records
FROM flight_delays
GROUP BY airport_name
HAVING COUNT(*) > 50
ORDER BY avg_delay_pct DESC
LIMIT 10;

-- best airports by lowest delay %
SELECT airport_name, ROUND(AVG(delay_rate)*100,2) AS avg_delay_pct, COUNT(*) AS total_records
FROM flight_delays
GROUP BY airport_name
HAVING COUNT(*) > 50
ORDER BY avg_delay_pct ASC
LIMIT 10;

-- top 10 most reliable airlines with lowest delay %
SELECT carrier_name, ROUND(AVG(delay_rate)*100, 2) AS avg_delay_pct
FROM flight_delays
GROUP BY carrier_name
HAVING COUNT(*) > 50
ORDER BY avg_delay_pct ASC
LIMIT 10;

-- delay trend by month for all airlines
SELECT year, month, ROUND(AVG(delay_rate)*100,2) AS avg_delay_pct
FROM flight_delays
GROUP BY year, month
ORDER BY year, month;

-- delay breakdown by cause
SELECT ROUND(SUM(carrier_delay) / SUM(arr_delay) * 100, 2) AS carrier_delay_pct,
ROUND(SUM(weather_delay) / SUM(arr_delay) * 100, 2) AS weather_delay_pct,
ROUND(SUM(nas_delay) / SUM(arr_delay) * 100, 2) AS nas_delay_pct,
ROUND(SUM(late_aircraft_delay) / SUM(arr_delay) * 100, 2) AS late_aircraft_delay_pct
FROM flight_delays;

-- where was the worst experience (airline and airport combo)
SELECT carrier_name, airport_name, ROUND(AVG(arr_delay), 2) AS avg_delay_pct, COUNT(*) AS flights
FROM flight_delays
GROUP BY carrier_name, airport_name
ORDER BY avg_delay_pct DESC LIMIT 10;



Writing queries.sql
